In [1]:
import pandas as pd
import plotly.graph_objects as go
from signals import *

In [ ]:
# Can we reasample dataframes?
print(load_ticker("AUDCAD").head(2))
print(load_ticker("AUDCAD", 60).head(2))
# Yes!

In [ ]:
# Can we calculate the supertrend?
from supertrend.Indicators.Super_Trend import ST
print("Loading data")
df = load_ticker("AUDCAD", 60)

print("Calculating SuperTrend")
df_60_ST = ST(df, 3, 10)[["Open","High","Low","Close","ATR","SuperTrend","SuperTrendType"]]
display(df_60_ST)
# Yes!

In [5]:
# How does indexing work in a resampled df
# Signal comes in at 03:30, so the 03:00 candle has not finished yet, we should use the 
# indicator from the previous candle, 02:00
#display(pd.concat([df_60.loc[:"2019-10-29 03:30"][-2:],df_60.loc["2019-10-29 03:30":]])) # -> Starts at 04:00, is this what I want

def trailing_df(df, datetime_str):
    """
    Given a signal within a candle, this function returns a df starting with the most recent 
    complete candle.
    
    e.g.
    1 hr sampling. 03:30 signal time (3-4 candle not yet complete), returns starting with the 02:00 candle
    """  
    return pd.concat([df.loc[:datetime_str][-2:],df.loc[datetime_str:]])
    

In [36]:
def supertrend_trailing(df_full, signal):
    """
    Process signal using trailing supertrend stop loss.
    TODO: accound for spread and fees
    """
    #display(df_full)
    df = trailing_df(df_full, signal.datetime_str)
    #display(df)

    real_entry = signal.entry_price # Assume trade entered immediatley
    stop_loss = signal.stop_loss
    
    ohlc_values = df.iterrows()
    _, previous_ohlc = next(ohlc_values)
    
    previous_type = previous_ohlc.SuperTrendType
    previous_supertrend = previous_ohlc.SuperTrend
    

    for dt, ohlc in ohlc_values:
        if signal.type == "BUY":
            if previous_type == 1:
                if stop_loss < previous_supertrend:
                    stop_loss = previous_supertrend
            
            if ohlc.High >= signal.take_profit:
                return signal.take_profit - real_entry
            if ohlc.Low <= stop_loss:
                return stop_loss - real_entry
            
        if signal.type == "SELL":
            if previous_type == -1:
                if stop_loss > previous_supertrend:
                    stop_loss = previous_supertrend   
                    
            if ohlc.Low <= signal.take_profit:
                return real_entry - signal.take_profit
            if ohlc.High >= stop_loss:
                return real_entry - stop_loss
            
        previous_type = ohlc.SuperTrendType
        previous_supertrend = ohlc.SuperTrend
        
    return 0.0
        

In [130]:
beta_tp1 = Signal(ticker="NZDCAD", datetime_str="2019-12-04 12:44",
                  entry_price=0.8656, stop_loss=0.868, take_profit=0.8636)
df = get_supertrend(beta_tp1.ticker, 6, 40, 60)

print(beta_tp1, supertrend_trailing(df, beta_tp1) * pip_factor(beta_tp1.ticker))

NZDCAD: 2019-12-04 12:44: Entry: 0.8656, SL: 0.868, TP: 0.8636 20.000000000000018


With SL, TP and Trailing loss
60mins
SL = 50 pips
TP = 70 pips
Length = 40
Multiplier = 6
profit = 1514 pips (30 wins, 22 losses)

30mins
SL = 50 pips
TP = 70 pips
Length = 40
Multiplier = 8
profit = 1430 pips (29 wins, 23 losses)

15mins
SL = 50 pips
TP = 70 pips
Length = 40
Multiplier = 13
profit = 1227 pips (27 wins, 25 losses)

In [129]:
# Multiplier Analysis

for multiplier in [11, 12, 13]:
    signals = load_beta_trades("./beta/BETA.csv")
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals["TP1"]:
        signal.set_stop_loss_pips(50)
        signal.set_take_profit_pips(70)

        df = get_supertrend(signal.ticker, multiplier=multiplier, length=40, sample_mins=15)
        profit = supertrend_trailing(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        # print(signal, profit)
    print(multiplier, total_profit, wins, losses)

11 1059.433554519316 29 23
12 1203.7949395596272 27 25
13 1227.3398707565373 27 25


In [126]:
# Length Analysis

for length in range(20, 120, 20):
    signals = load_beta_trades("./beta/BETA.csv")
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals["TP1"]:
        signal.set_stop_loss_pips(50)
        signal.set_take_profit_pips(70)
        df = get_supertrend(signal.ticker, multiplier=8, length=length, sample_mins=30)
        profit = supertrend_trailing(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    print(length, total_profit, wins, losses)

20 1198.5987056670417 27 25
40 1430.2646701937472 29 23
60 1300.8804053538481 28 24
80 1298.4554808567623 28 24
100 1300.020551115588 28 24


In [ ]:
# Example plotly scatter plot
fig = go.Figure(data=go.Scatter(x=list(take_profits), y=profits))
fig.update_layout(
    title="BETA Take Profit Optimisation",
    title_x=0.5,
    xaxis_title="Take Profit Level (pips)",
    yaxis_title="Profit (pips)"
)
fig.show()

In [ ]:
# Example plotly candlestick plot
dfp = df.loc["2019-01-14 21:00":"2019-01-14 22:00"]
fig = go.Figure(data=[go.Candlestick(x=dfp.index,
                open=dfp['Open'],
                high=dfp['High'],
                low=dfp['Low'],
                close=dfp['Close'])])

fig.show()

In [ ]:
# How to pretty print a dataframe
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_60)